# VGG-like model for the CIFAR-10 dataset

In this notebook, I present a model able to reach 90 % accuracy on test set by using data augmentation and using an architecture inspired by the VGG model.

- Karen Simonyan, Andrew Zisserman, 2015, Very Deep Convolutional Networks for Large-Scale Image Recognition

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint

import time

/home/arthur/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Preparation

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('X shape :', x_train.shape)
print(len(x_train), 'train samples')
print(len(x_test), 'test samples')

X shape : (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_train_mean)/x_train_std

n_y = 10
y_train = keras.utils.to_categorical(y_train, n_y)
y_test = keras.utils.to_categorical(y_test, n_y)

# Model

In [4]:
input_shape = x_train.shape[1:]
activation = 'relu'
l2_reg = keras.regularizers.l2(0.0001)

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=input_shape, kernel_regularizer=l2_reg, 
                 activation=activation))
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2_reg, activation=activation))
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2_reg, activation=activation))
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2_reg, activation=activation))
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.25))
          
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2_reg, activation=activation))
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2_reg, activation=activation))
model.add(AveragePooling2D((8,8)))
model.add(Dropout(0.25))
          
model.add(Flatten())
model.add(Dense(128, activation=activation))
model.add(Dropout(0.25))
model.add(Dense(n_y, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
__________

# Model training

In [5]:
optimizer = keras.optimizers.SGD(0.01, momentum=0.9, nesterov=True)
model.compile(optimizer, keras.losses.categorical_crossentropy, ['accuracy'])

shift = 4/32
generator = ImageDataGenerator(width_shift_range=shift, height_shift_range=shift, 
                               horizontal_flip=True)

batch_size = 64
n_steps = x_train.shape[0]//batch_size

save_path = './Model_trained/VGG_cifar10.h5'
ckeckpoint = ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=np.sqrt(0.1), patience=5, 
                                              min_delta=0.01, min_lr=1e-6, verbose=1)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=20, verbose=1)

t0 = time.time()
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=n_steps, 
                    epochs=200, validation_data=(x_test, y_test), callbacks=[ckeckpoint, reduce_lr, early_stopping])
print('Total training time : %.3f s' %(time.time()-t0))

Epoch 1/200
781/781 [==============================] - 18s 22ms/step - loss: 1.9902 - acc: 0.2365 - val_loss: 1.6864 - val_acc: 0.3555

Epoch 00001: val_acc improved from -inf to 0.35550, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 2/200
781/781 [==============================] - 16s 21ms/step - loss: 1.6788 - acc: 0.3680 - val_loss: 1.6967 - val_acc: 0.3956

Epoch 00002: val_acc improved from 0.35550 to 0.39560, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 3/200
781/781 [==============================] - 16s 21ms/step - loss: 1.5278 - acc: 0.4424 - val_loss: 1.3621 - val_acc: 0.5145

Epoch 00003: val_acc improved from 0.39560 to 0.51450, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 4/200
781/781 [==============================] - 16s 21ms/step - loss: 1.3799 - acc: 0.5096 - val_loss: 1.2298 - val_acc: 0.5573

Epoch 00004: val_acc improved from 0.51450 to 0.55730, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 5/200
781/781 [==============================

781/781 [==============================] - 16s 21ms/step - loss: 0.6121 - acc: 0.8254 - val_loss: 0.5937 - val_acc: 0.8371

Epoch 00039: val_acc did not improve from 0.83940
Epoch 40/200
781/781 [==============================] - 16s 21ms/step - loss: 0.6052 - acc: 0.8292 - val_loss: 0.5811 - val_acc: 0.8405

Epoch 00040: val_acc improved from 0.83940 to 0.84050, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 41/200
781/781 [==============================] - 16s 21ms/step - loss: 0.6034 - acc: 0.8291 - val_loss: 0.5895 - val_acc: 0.8369

Epoch 00041: val_acc did not improve from 0.84050
Epoch 42/200
781/781 [==============================] - 16s 21ms/step - loss: 0.5959 - acc: 0.8329 - val_loss: 0.5877 - val_acc: 0.8372

Epoch 00042: val_acc did not improve from 0.84050
Epoch 43/200
781/781 [==============================] - 16s 20ms/step - loss: 0.5948 - acc: 0.8331 - val_loss: 0.6137 - val_acc: 0.8372

Epoch 00043: val_acc did not improve from 0.84050
Epoch 44/200
781/781 [====

781/781 [==============================] - 16s 21ms/step - loss: 0.4323 - acc: 0.8897 - val_loss: 0.4783 - val_acc: 0.8805

Epoch 00079: val_acc improved from 0.87930 to 0.88050, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 80/200
781/781 [==============================] - 16s 21ms/step - loss: 0.4314 - acc: 0.8901 - val_loss: 0.4745 - val_acc: 0.8810

Epoch 00080: val_acc improved from 0.88050 to 0.88100, saving model to ./Model_trained/VGG_cifar10.h5
Epoch 81/200
781/781 [==============================] - 16s 21ms/step - loss: 0.4283 - acc: 0.8898 - val_loss: 0.4799 - val_acc: 0.8803

Epoch 00081: val_acc did not improve from 0.88100

Epoch 00081: ReduceLROnPlateau reducing learning rate to 0.00010000000639606199.
Epoch 82/200
781/781 [==============================] - 16s 21ms/step - loss: 0.4269 - acc: 0.8912 - val_loss: 0.4916 - val_acc: 0.8758

Epoch 00082: val_acc did not improve from 0.88100
Epoch 83/200
781/781 [==============================] - 16s 21ms/step - loss: 0

# Final evaluation

In [6]:
best_model = keras.models.load_model(save_path)
print('Best model test accuracy :', best_model.evaluate(x_test, y_test, batch_size=64)[1])

10000/10000 [==============================] - 1s 102us/step
Best model test accuracy : 0.881


# Conclusion

Building a deep ConvNet, inspired by the VGG-Net, and using dropout, let us reach $88.1 \%$ of accuracy on the CIFAR-10 dataset. The use of dropout let us restrict the overfitting of the model, despite his high number of parameters.

The closeness between the training and testing loss and the low training accuracy, let us think that increasing a bit more the depth would lead to increased accuracy results.